# Learn the Druid Python API

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

This notebook provides a quick introduction to the Python wrapper around the [Druid REST API](api-tutorial.ipynb). This notebook assumes you are familiar with the basics of the REST API, and the [set of operations which Druid provides](https://druid.apache.org/docs/latest/operations/api-reference.html). This tutorial focuses on using Python to access those APIs rather than explaining the APIs themselves. The APIs themselves are covered in other notebooks that use the Python API.

The Druid Python API is primarily intended to help with these notebook tutorials. It can also be used in your own ad-hoc notebooks, or in a regular Python program.

The Druid Python API is a work in progress. The Druid team adds API wrappers as needed for the notebook tutorials. If you find you need additional wrappers, please feel free to add them, and post a PR to Apache Druid with your additions.

The API provides two levels of functions. Most are simple wrappers around Druid's REST APIs. Others add additional code to make the API easier to use. The SQL query interface is a prime example: extra code translates a simple SQL query into Druid's `SQLQuery` object and interprets the results into a form that can be displayed in a notebook.

This notebook contains sample output to allow it to work a bit like a reference. To run it yourself, start by using the `Kernel` &rarr; `Restart & Clear Output` menu command to clear the sample output.

Start by importing the `druidapi` package from the same folder as this notebook. The `styles()` calls adds some CSS styles needed to display results.

In [1]:
import druidapi
druidapi.styles()

Next, connect to your cluster by providing the router endpoint. The code assumes the cluster is on your local machine, using the default port. Go ahead and change this if your setup is different.

The API uses the router to forward messages to each of Druid's services so that you don't have to keep track of the host and port for each service.

In [2]:
druid = druidapi.client('http://localhost:8888')

## Status Client

The SDK groups Druid REST API calls into categories, with a client for each. Start with the status client.

In [3]:
status_client = druid.status()

Use the Python `help()` function to learn what methods are avaialble.

In [4]:
help(status_client)

Help on StatusClient in module druidapi.status object:

class StatusClient(builtins.object)
 |  StatusClient(rest_client)
 |  
 |  Client for status APIs. These APIs are available on all nodes.
 |  If used with the router, they report the status of just the router.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, rest_client)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  brokers(self)
 |  
 |  in_cluster(self)
 |      Returns `True` if the node is visible wihtin the cluster, `False` if not.
 |      (That is, returns the value of the `{"selfDiscovered": true/false}`
 |      field in the response.
 |      
 |      GET `/status/selfDiscovered/status`
 |      
 |      See https://druid.apache.org/docs/latest/operations/api-reference.html#process-information
 |  
 |  is_healthy(self) -> bool
 |      Returns `True` if the node is healthy, an exception otherwise.
 |      Useful for automated health checks.
 |      
 |      GET `/status/health`
 |      


Druid servers return unexpected results if you make REST calls while Druid starts up. The following will run until the server is ready. If you forgot to start your server, or the URL above is wrong, this will hang forever. Use the Kernel &rarr; Interrupt command to break out of the function. (Or, start your server. If your server refuses to start, then this Jupyter Notebook may be running on port 8888. See the [README](README.md) for how to start on a different port.)

In [5]:
status_client.wait_until_ready()

Check the version of your cluster. Some of these notebooks illustrate newer features available only on specific versions of Druid.

In [6]:
status_client.version()

'26.0.0-SNAPSHOT'

You can also check which extensions are loaded in your cluster. Some notebooks require specific extensions to be available.

In [7]:
status_client.properties()['druid.extensions.loadList']

'["druid-hdfs-storage", "druid-kafka-indexing-service", "druid-datasketches", "druid-multi-stage-query", "druid-lookups-cached-global", "druid-catalog"]'

## SQL Client

Running SQL queries in a notebook is easy. Here is an example of how to run a query and display results. The [pydruid](https://pythonhosted.org/pydruid/) library provides a robust way to run native queries, to run SQL queries, and to convert the results to various formats. Here the goal is just to interact with Druid.

In [8]:
sql_client = druid.sql()

Start by getting a list of schemas.

In [10]:
sql_client.show_schemas()

SchemaName
INFORMATION_SCHEMA
druid
ext
lookup
sys
view


Then, retreive the tables (or datasources) within any schema.

In [11]:
sql_client.show_tables('INFORMATION_SCHEMA')

TableName
COLUMNS
PARAMETERS
SCHEMATA
TABLES


The above shows the list of datasources by default. You'll get an empty result if you have no datasources yet.

In [12]:
sql_client.show_tables()

TableName
myWiki
myWiki3


You can easily run a query and show the results:

In [14]:
sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'INFORMATION_SCHEMA'
'''
sql_client.show(sql)

TABLE_NAME
COLUMNS
PARAMETERS
SCHEMATA
TABLES


The query above showed the same results as `show_tables()`. That is not surprising: `show_tables()` just runs this query for you.

The API also allows passing context parameters and query parameters using a request object. Druid will work out the query parameter type based on the Python type. Pass context values as a Python `dict`.

In [18]:
sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = ?
'''
req = sql_client.sql_request(sql)
req.add_parameter('INFORMATION_SCHEMA')
req.with_context({"someParameter": "someValue"})
sql_client.show(req)

TABLE_NAME
COLUMNS
PARAMETERS
SCHEMATA
TABLES


The request has other features for advanced use cases: see the code for details. The query API actually returns a sql response object. Use this if you want to get the values directly, work with the schema, etc.

In [20]:
sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'INFORMATION_SCHEMA'
'''
resp = sql_client.sql_query(sql)

In [29]:
col1 = resp.schema()[0]
print(col1.name, col1.sql_type, col1.druid_type)

TABLE_NAME VARCHAR string


In [24]:
resp.rows()

[{'TABLE_NAME': 'COLUMNS'},
 {'TABLE_NAME': 'PARAMETERS'},
 {'TABLE_NAME': 'SCHEMATA'},
 {'TABLE_NAME': 'TABLES'}]

The `show()` method uses this information for format an HTML table to present the results.

## MSQ Ingestion

The SQL client also performs MSQ-based ingestion using `INSERT` or `REPLACE` statements. Use the extension check above to ensure that `druid-multi-stage-query` is loaded in Druid 26. (Later versions may have MSQ built in.)

An MSQ query is run using a different API: `task()`. This API returns a response object that describes the Overlord task which runs the MSQ query. For tutorials, data is usually small enough you can wait for the ingestion to complete. Do that with the `run_task()` call which handles the waiting. To illustrate, here is a query that ingests a subset of columns, and includes a few data clean-up steps:

In [31]:
sql = '''
REPLACE INTO "myWiki1" OVERWRITE ALL
SELECT
  TIME_PARSE("timestamp") AS "__time",
  namespace,
  page,
  channel,
  "user",
  countryName,
  CASE WHEN isRobot = 'true' THEN 1 ELSE 0 END AS isRobot,
  "added",
  "delta",
  CASE WHEN isNew = 'true' THEN 1 ELSE 0 END AS isNew,
  CAST("deltaBucket" AS DOUBLE) AS deltaBucket,
  "deleted"
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}',
    '[{"name":"isRobot","type":"string"},{"name":"channel","type":"string"},{"name":"timestamp","type":"string"},{"name":"flags","type":"string"},{"name":"isUnpatrolled","type":"string"},{"name":"page","type":"string"},{"name":"diffUrl","type":"string"},{"name":"added","type":"long"},{"name":"comment","type":"string"},{"name":"commentLength","type":"long"},{"name":"isNew","type":"string"},{"name":"isMinor","type":"string"},{"name":"delta","type":"long"},{"name":"isAnonymous","type":"string"},{"name":"user","type":"string"},{"name":"deltaBucket","type":"long"},{"name":"deleted","type":"long"},{"name":"namespace","type":"string"},{"name":"cityName","type":"string"},{"name":"countryName","type":"string"},{"name":"regionIsoCode","type":"string"},{"name":"metroCode","type":"long"},{"name":"countryIsoCode","type":"string"},{"name":"regionName","type":"string"}]'
  )
)
PARTITIONED BY DAY
CLUSTERED BY namespace, page
'''

In [32]:
sql_client.run_task(sql)

MSQ reports task completion as soon as ingestion is done. However, it takes a while for Druid to load the resulting segments. Wait for the table to become ready.

In [34]:
sql_client.wait_until_ready('myWiki1')

`describe_table()` lists the columns in a table.

In [35]:
sql_client.describe_table('myWiki1')

Position,Name,Type
1,__time,TIMESTAMP
2,namespace,VARCHAR
3,page,VARCHAR
4,channel,VARCHAR
5,user,VARCHAR
6,countryName,VARCHAR
7,isRobot,BIGINT
8,added,BIGINT
9,delta,BIGINT
10,isNew,BIGINT


You can sample a few rows of data.

In [36]:
sql_client.show('SELECT * FROM myWiki1 LIMIT 10')

__time,namespace,page,channel,user,countryName,isRobot,added,delta,isNew,deltaBucket,deleted
2016-06-27T00:00:11.080Z,Main,Salo Toraut,#sv.wikipedia,Lsjbot,,1,31,31,1,0.0,0
2016-06-27T00:00:17.457Z,利用者,利用者:ワーナー成増/放送ウーマン賞,#ja.wikipedia,ワーナー成増,,0,125,125,0,100.0,0
2016-06-27T00:00:34.959Z,Main,Bailando 2015,#en.wikipedia,181.230.118.178,Argentina,0,2,2,0,0.0,0
2016-06-27T00:00:36.027Z,Main,Richie Rich's Christmas Wish,#en.wikipedia,JasonAQuest,,0,0,-2,0,-100.0,2
2016-06-27T00:00:46.874Z,Main,"El Olivo, Ascensión",#sh.wikipedia,Kolega2357,,1,0,-1,0,-100.0,1
2016-06-27T00:00:56.913Z,Main,Blowback (intelligence),#en.wikipedia,Brokenshardz,,0,76,76,0,0.0,0
2016-06-27T00:00:58.599Z,Kategoria,Kategoria:Dyskusje nad usunięciem artykułu zakończone bez konsensusu − lipiec 2016,#pl.wikipedia,Beau.bot,,1,270,270,1,200.0,0
2016-06-27T00:01:01.364Z,Main,"El Paraíso, Bachíniva",#sh.wikipedia,Kolega2357,,1,0,-1,0,-100.0,1
2016-06-27T00:01:03.685Z,Main,"El Terco, Bachíniva",#sh.wikipedia,Kolega2357,,1,0,-1,0,-100.0,1
2016-06-27T00:01:07.347Z,Main,Neqerssuaq,#ceb.wikipedia,Lsjbot,,1,4150,4150,1,4100.0,0


## Datasource Client

The Datasource client lets you perform operations on datasource objects. The SQL layer allows you to get metadata and do queries. The datasource client works with the underlying segments. Explaining the full functionality is the topic of another notebook. For now, you can use the datasource client to clean up the datasource created above. The `True` argument asks for "if exists" semantics so you don't get an error if the datasource was alredy deleted.

In [37]:
ds_client = druid.datasources()
ds_client.drop('myWiki', True)

## Tasks Client

Use the tasks client to work with Overlord tasks. The `run_task()` call above actually uses the task client internally to poll Overlord.

In [40]:
task_client = druid.tasks()
task_client.tasks()

[{'id': 'query-24066a63-7e20-41bb-b212-80f193e6f2c8-worker0_0',
  'groupId': 'query-24066a63-7e20-41bb-b212-80f193e6f2c8',
  'type': 'query_worker',
  'createdTime': '2023-02-09T22:49:01.761Z',
  'queueInsertionTime': '1970-01-01T00:00:00.000Z',
  'statusCode': 'SUCCESS',
  'status': 'SUCCESS',
  'runnerStatusCode': 'NONE',
  'duration': 57895,
  'location': {'host': 'localhost', 'port': 8101, 'tlsPort': -1},
  'dataSource': 'myWiki1',
  'errorMsg': None},
 {'id': 'query-24066a63-7e20-41bb-b212-80f193e6f2c8',
  'groupId': 'query-24066a63-7e20-41bb-b212-80f193e6f2c8',
  'type': 'query_controller',
  'createdTime': '2023-02-09T22:48:30.512Z',
  'queueInsertionTime': '1970-01-01T00:00:00.000Z',
  'statusCode': 'SUCCESS',
  'status': 'SUCCESS',
  'runnerStatusCode': 'NONE',
  'duration': 92476,
  'location': {'host': 'localhost', 'port': 8100, 'tlsPort': -1},
  'dataSource': 'myWiki1',
  'errorMsg': None}]

## REST Client

The Druid Python API starts with a REST client that itself is built on the `requests` package. The REST client implements the common patterns seen in the Druid REST API. You can create a client directly:

In [45]:
from druidapi.rest import DruidRestClient
rest_client = DruidRestClient("http://localhost:8888")

Or, if you have already created the Druid client, you can reuse the existing REST client. This is how the various other clients work internally.

In [ ]:
rest_client = druid.rest()

Use the REST client if you need to make calls that are not yet wrapped by the Python API, or if you want to do something special. To illustrate the client, you can make some of the same calls as in the [Druid REST API notebook](api_tutorial.ipynb).

The REST API maintains the Druid host: you just provide the specifc URL tail. There are methods to get or post JSON results. For example, to get status information:

In [53]:
rest_client.get_json('/status')

{'version': '26.0.0-SNAPSHOT',
 'modules': [{'name': 'org.apache.druid.common.aws.AWSModule',
   'artifact': 'druid-aws-common',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.common.gcp.GcpModule',
   'artifact': 'druid-gcp-common',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.storage.hdfs.HdfsStorageDruidModule',
   'artifact': 'druid-hdfs-storage',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.indexing.kafka.KafkaIndexTaskModule',
   'artifact': 'druid-kafka-indexing-service',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.query.aggregation.datasketches.theta.SketchModule',
   'artifact': 'druid-datasketches',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.query.aggregation.datasketches.theta.oldapi.OldApiSketchModule',
   'artifact': 'druid-datasketches',
   'version': '26.0.0-SNAPSHOT'},
  {'name': 'org.apache.druid.query.aggregation.datasketches.quantiles.DoublesSketchModule',
   'artifact': 'druid

A quick comparison of the three approaches (Requests, REST client, Python client):

Status:
* Requests: `session.get(druid_host + '/status').json()`
* REST client: `rest_client.get_json('/status')`
* Status client: `status_client.status()`

Health:
* Requests: `session.get(druid_host + '/status/health').json()`
* REST client: `rest_client.get_json('/status/health')`
* Status client: `status_client.is_healthy()`

Ingest data:
* Requests: See the [REST tutorial](api_tutorial.ipynb)
* REST client: as the REST tutorial, but use `rest_client.post_json('/druid/v2/sql/task', sql_request)` and
  `rest_client.get_json(f"/druid/indexer/v1/task/{ingestion_taskId}/status")`
* SQL client: `sql_client.run_task(sql)`, also a form for a full SQL request.

List datasources:
* Requests: `session.get(druid_host + '/druid/coordinator/v1/datasources').json()`
* REST client: `rest_client.get_json('/druid/coordinator/v1/datasources')`
* Datasources client: `ds_client.names()`

Query data, where `sql_request` is a properly-formatted `SqlResquest` dictionary:
* Requests: `session.post(druid_host + '/druid/v2/sql', json=sql_request).json()`
* REST client: `rest_client.post_json('/druid/v2/sql', sql_request)`
* SQL Client: `sql_client.show(sql)`, where `sql` is the query text

In general, you have to provide the all the details for the Requests library. The REST client handles the low-level repetitious bits. The Python clients provide methods that encapsulate the specifics of the URLS and return formats.

## Constants

Druid has a large number of special constants: type names, options, etc. The `consts` module provides definitions for many of these:

In [38]:
from druidapi import consts

In [39]:
help(consts)

Help on module druidapi.consts in druidapi:

NAME
    druidapi.consts

DESCRIPTION
    # Licensed to the Apache Software Foundation (ASF) under one or more
    # contributor license agreements.  See the NOTICE file distributed with
    # this work for additional information regarding copyright ownership.
    # The ASF licenses this file to You under the Apache License, Version 2.0
    # (the "License"); you may not use this file except in compliance with
    # the License.  You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.

DATA
    COLUMNS_TABLE = 'INFORMATION_SCHEMA.COLUMNS'
    COORD_BASE = '/druid/

Using the constants avoids typos:

In [41]:
sql_client.show_tables(consts.SYS_SCHEMA)

TableName
segments
server_segments
servers
supervisors
tasks


## Tracing

It is often handy to see what the Druid API is doing: what messages it sends to Druid. You may need to debug some function that isn't working as expected. Or, perhaps you want to see what is sent to Druid so you can replicate it in your own code. Either way, just turn on tracing:

In [42]:
druid.trace(True)

Then, each call to Druid prints what it sends:

In [43]:
sql_client.show_tables()


            SELECT TABLE_NAME AS TableName
            FROM INFORMATION_SCHEMA.TABLES
            WHERE TABLE_SCHEMA = 'druid'
            ORDER BY TABLE_NAME
            
POST: http://localhost:8888/druid/v2/sql
body: {'query': "\n            SELECT TABLE_NAME AS TableName\n            FROM INFORMATION_SCHEMA.TABLES\n            WHERE TABLE_SCHEMA = 'druid'\n            ORDER BY TABLE_NAME\n            ", 'resultFormat': 'object'}


TableName
myWiki
myWiki1
myWiki3


## Conclusion

This notebook have you a whirlwind tour of the Python Druid API: just enough to check your cluster, ingest some data with MSQ and query that data. Druid has many more APIs. As noted earlier, the Python API is a work in progress: the team adds new wrappers as needed for tutorials. Your [contributions](https://github.com/apache/druid/pulls) and [feedback](https://github.com/apache/druid/issues) are welcome.